In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import os 
import re
with open('Tiga.csv', mode='a',newline='') as csv_file:
    fieldnames = ['price', 'square','floors','rooms','subway', 'area']
    # 设置字段
    writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    #writer.writeheader(); #写入csv标题,第一次爬数据时使用

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36"}
    url = 'https://cq.58.com/ershoufang/sub/pn2'#pnXX 指第x页
    r = requests.get(url=url, headers=headers)
    html = r.text        
    # html.parser解析器
    soup = BeautifulSoup(html, 'html.parser')
    con = soup.find('ul',class_='house-list-wrap')
    con_list = con.find_all('li' ,class_= 'sendsoj')
    con_list
    
    dict = {'奉节县':26,'巴南区':1,'北碚区': 2,'璧山区': 3,'大渡口区': 4,'大足区':5,'垫江县':6,'涪陵区':7,
            '合川区':8,'江北区':9,'江津区':10,'九龙坡区':11,'开州区(开县)':12,'梁平县':13,'南岸区':14,'南川区':15,
            '荣昌区':16,'沙坪坝区':17,'铜梁区':18,'潼南区':19,'万州区':20,'永川区':21,'渝北区':22,'渝中区':23,'云阳县':24,'长寿区':25}#设置字典,通过数据来设定特征值
    
    for i in con_list:
        name = i.find('h2',class_='title').find('a').get_text();
        price1 = i.find('div',class_='price').find('p',class_ = 'unit').get_text();
        price = re.compile('^[0-9]*').findall(price1)[0]
        list = i.find('div',class_='list-info').find('p',class_ = 'baseinfo').find_all('span');
        square2 = list[1].get_text();
        square = re.compile('[1-9]\d*\.\d*|0\.\d*[1-9]\d*').findall(square2)[0]
        floors1 = list[3].get_text();
        floors = re.compile('[0-9]+').findall(floors1)[0]
        rooms1 = list[0].get_text();
        rooms = int(re.compile('[0-9]+').findall(rooms1)[0]) + int(re.compile('[0-9]+').findall(rooms1)[1])+ int(re.compile('[0-9]+').findall(rooms1)[2])
        subwaybox = i.find('div',class_='list-info').find_all('p',class_ = 'baseinfo')[1].find_all('span');
        subway1 = "";
        if len(subwaybox)>1 :
            subway1 = subwaybox[1].get_text();
        else :
            subway1 = "";
        subway = 0;
        if len(subway1)>1:
            subway = 1;
        
        area1= i.find('div',class_='list-info').find_all('p',class_ = 'baseinfo')[1].find_all('a')[1].get_text();
        area = dict[area1]
        json = {'price':price ,'square':square,'floors':floors,'rooms' : rooms,'subway': subway , 'area': area}
        print(json)#打印结果
        writer.writerow(json)#将结果写入csv


{'price': '14125', 'square': '63.01', 'floors': '25', 'rooms': 5, 'subway': 1, 'area': 14}
{'price': '8487', 'square': '74.0', 'floors': '32', 'rooms': 5, 'subway': 0, 'area': 21}
{'price': '15802', 'square': '63.92', 'floors': '33', 'rooms': 5, 'subway': 1, 'area': 17}
{'price': '10318', 'square': '126.0', 'floors': '15', 'rooms': 8, 'subway': 1, 'area': 14}
{'price': '8899', 'square': '118.0', 'floors': '6', 'rooms': 8, 'subway': 1, 'area': 2}
{'price': '13379', 'square': '79.23', 'floors': '46', 'rooms': 5, 'subway': 1, 'area': 17}
{'price': '4337', 'square': '101.0', 'floors': '8', 'rooms': 7, 'subway': 0, 'area': 6}
{'price': '7541', 'square': '61.0', 'floors': '30', 'rooms': 5, 'subway': 0, 'area': 10}
{'price': '5758', 'square': '99.0', 'floors': '32', 'rooms': 7, 'subway': 0, 'area': 10}
{'price': '15704', 'square': '146.46', 'floors': '25', 'rooms': 7, 'subway': 1, 'area': 14}
{'price': '11643', 'square': '140.0', 'floors': '8', 'rooms': 8, 'subway': 1, 'area': 22}
{'price': '

KeyError: '丰都县'